## Import dependencies

In [36]:
# Utils
import uuid
import json
import time
import uuid
from typing import List
import numpy as np
import textwrap



# Vertex AI
import vertexai
from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")



# Langchain
import langchain

print(f"LangChain version: {langchain.__version__}")

from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI


# Import custom Matching Engine packages
from utils.matching_engine import MatchingEngine
from utils.matching_engine_utils import MatchingEngineUtils



Vertex AI SDK version: 1.27.0
LangChain version: 0.0.201


In [37]:
PROJECT_ID = "analytics-ml-ai"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
ME_DIMENSIONS = 768 # when using Vertex PaLM Embedding
ME_DISPLAY_NAME = "rfpbot_all_products_stage"
# ME_NAME = "rfpbot_all_products_stage"
ME_DESCRIPTION = "rfpbot across all products stage"
ME_EMBEDDING_DIR   = "gs://rfpbot-stage-me" # @param {type:"string"}
GCS_BUCKET="gs://rfpbot-stage-me"

In [38]:
# Authenticate with Google Cloud credentials
# from google.colab import auth as google_auth
# google_auth.authenticate_user()

import logging
logging.basicConfig(level = logging.INFO)

import numpy as np

## LangChain Wrappers and Utilities

**Run the following cells**

## Initializing LLMs with LangChain & Matching Engine as Vector Database

In [25]:
mengine = MatchingEngineUtils(PROJECT_ID, LOCATION, ME_DISPLAY_NAME)

In [26]:
ME_INDEX_ID, ME_INDEX_ENDPOINT_ID = mengine.get_index_and_endpoint()
print(f"ME_INDEX_ID={ME_INDEX_ID}")
print(f"ME_INDEX_ENDPOINT_ID={ME_INDEX_ENDPOINT_ID}")


ME_INDEX_ID=projects/184378960328/locations/us-central1/indexes/9057504110734999552
ME_INDEX_ENDPOINT_ID=projects/184378960328/locations/us-central1/indexEndpoints/7247057060532060160


In [27]:
REQUESTS_PER_MINUTE = 300


llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=512,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Chat instance integrated with langChain
#chat = VertexChat()

# Embeddings API integrated with langChain
embedding = VertexAIEmbeddings(requests_per_minute=REQUESTS_PER_MINUTE)

embedding

VertexAIEmbeddings(client=<vertexai.language_models._language_models._PreviewTextEmbeddingModel object at 0x7efbe0c09db0>, model_name='textembedding-gecko', temperature=0.0, max_output_tokens=128, top_p=0.95, top_k=40, stop=None, project=None, location='us-central1', credentials=None)

In [28]:
me = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=LOCATION,
    gcs_bucket_name=GCS_BUCKET,
    embedding=embedding,
    index_id=ME_INDEX_ID,
    endpoint_id=ME_INDEX_ENDPOINT_ID
)

---

## Question/Answering Chain

LangChain provides easy ways to chain multiple tasks that can do QA over a set of documents, called QA chains. We use [**RetrievalQA**](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) chain actually uses **load_qa_chain** under the hood.

#### Approach # 1

In [29]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Please be verbose

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", 
    retriever=me.as_retriever(), 
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True)

In [30]:
query = "describe your internal controls for backup and recovery"

result = qa({"query": query})
result["result"]

"Google has implemented business continuity measures to maintain the availability of Google's production infrastructure and services. Google has defined the risks and recovery objectives, as input, to establish the BCP. Google has established a framework to develop and maintain business continuity, and assigned each area to the executives within respective departments. Google conducts disaster recovery testing on an annual basis to provide a coordinated venue for infrastructure and application teams to test communication plans, failover scenarios, operational transition, and other emergency responses. Teams that participate in the disaster recovery exercise develop testing plans and postmortems which document the results and lessons learned from the tests."

#### Approach # 2

In [46]:
# Create chain to answer questions
NUMBER_OF_RESULTS = 10
SEARCH_DISTANCE_THRESHOLD = 0.6

# Expose index to the retriever
retriever = me.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": NUMBER_OF_RESULTS,
        "search_distance": SEARCH_DISTANCE_THRESHOLD,
    },
)

In [47]:
template = """SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: {question}

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

=============
{context}
=============

Question: {question}
Helpful Answer:"""

In [48]:
# Uses LLM to synthesize results from the search index.
# Use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    },
)

In [49]:
def formatter(result):
    print(f"Query: {result['query']}")
    print("." * 80)
    if "source_documents" in result.keys():
        for idx, ref in enumerate(result["source_documents"]):
            print("-" * 80)
            print(f"REFERENCE #{idx}")
            print("-" * 80)
            if "score" in ref.metadata:
                print(f"Matching Score: {ref.metadata['score']}")
            if "source" in ref.metadata:
                print(f"Document Source: {ref.metadata['source']}")
            if "document_name" in ref.metadata:
                print(f"Document Name: {ref.metadata['document_name']}")
            print("." * 80)
            print(f"Content: \n{wrap(ref.page_content)}")
    print("." * 80)
    print(f"Response: {wrap(result['result'])}")
    print("." * 80)


def wrap(s):
    return "\n".join(textwrap.wrap(s, width=120, break_long_words=False))


def ask(query, qa=qa, k=NUMBER_OF_RESULTS, search_distance=SEARCH_DISTANCE_THRESHOLD):
    qa.retriever.search_kwargs["search_distance"] = search_distance
    qa.retriever.search_kwargs["k"] = k
    result = qa({"query": query})
    return formatter(result)

In [50]:
ask("describe your internal controls for backup and recovery")



> Entering new  chain...

> Finished chain.
Query: describe your internal controls for backup and recovery
................................................................................
--------------------------------------------------------------------------------
REFERENCE #0
--------------------------------------------------------------------------------
Matching Score: 0.697128176689148
Document Source: ../sources/GCP-[Winter-2023] GCP SOC 2..pdf
................................................................................
Content: 
Google LLC | Description of Criteria, Controls, Tests and Results of Tests  149 Criteria, Controls, Tests and Results of
Tests  Control Description  SOC 2 Criteria Reference  Tests Performed by EY  Results  Inspected the documentation and
determined BCP and Disaster Recovery (DR) were tested on interval basis or upon significant organizational or
environmental changes.  No deviations noted.  Inspected a sample team guideline available and determ